<a href="https://colab.research.google.com/github/paruldiwakar/ML-DL/blob/master/Emoji_Prediction(LSTM_RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install emoji

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=a0f964eb01cbab713a241e74bb1339788a951493d06dc257ca65135010a0cc9e
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji

In [0]:
emoji.EMOJI_UNICODE.get(':baseball:')

'⚾'

In [0]:
emoji_dict = {
             "0":":growing_heart:",
             "1": ":baseball:",
              "2":":grinning_face_with_big_eyes:",
              "3":":disappointed_face:",
              "4":":fork_and_knife:",
              "5":":hundred_points:",
              "6":":fire:",
              "7":":face_blowing_a_kiss:",
              "8":":chestnut:",
              "9":":flexed_biceps:"

}

In [0]:
for e in emoji_dict.values():
  print(emoji.emojize(e))

💗
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [0]:
import pandas as pd
import numpy as np

In [0]:
!mkdir best_model

In [0]:
train = pd.read_csv("/train_emoji.csv",header=None)
test = pd.read_csv("/test_emoji.csv",header=None)

In [0]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a very nice raise\t,2
3,she got me a nice present\t,2
4,ha ha ha it was so funny\t,2


In [0]:
data = train.values
data

array([['never talk to me again', 3, nan, nan],
       ['I am proud of your achievements', 2, nan, nan],
       ['It is the worst day in my life', 3, nan, nan],
       ['Miss you so much', 0, nan, ' [0]'],
       ['food is life', 4, nan, nan],
       ['I love you mum', 0, nan, nan],
       ['Stop saying bullshit', 3, nan, nan],
       ['congratulations on your acceptance', 2, nan, nan],
       ['The assignment is too long ', 3, nan, nan],
       ['I want to go play', 1, nan, ' [3]'],
       ['she did not answer my text ', 3, nan, nan],
       ['Your stupidity has no limit', 3, nan, nan],
       ['how many points did he score', 1, nan, nan],
       ['my algorithm performs poorly', 3, nan, nan],
       ['I got approved', 2, nan, nan],
       ['Stop shouting at me', 3, nan, nan],
       ['Sounds like a fun plan ha ha', 2, nan, nan],
       ['no one likes him', 3, nan, nan],
       ['the game just finished', 1, nan, ' [2]'],
       ['I will celebrate soon', 2, nan, nan],
       ['So sad yo

In [0]:
X_train = train[0]
y_train = train[1]
X_test = test[0]
y_test = test[1]
#X_train[0] =(" ").join(X_train[0])

In [0]:
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=5)
y_test = to_categorical(y_test,num_classes=5)

In [0]:
y_train.shape,y_test.shape

((132, 5), (56, 5))

In [0]:
for i in range(5):
  print(X_train[i],emoji.emojize(emoji_dict[str(y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much 💗
food is life 🍴


In [0]:
f = open('/glove.6B.50d.txt',encoding='utf-8')

In [0]:
embeddings_index = {}

for line in f:
  values = line.split()
  word = values[0]
  coef = np.asarray(values[1:],dtype='float')

  embeddings_index[word] = coef

f.close() 

In [0]:
emb_dim = embeddings_index["eat"].shape[0]
emb_dim

50

### Converting sentences into vectors (like output of embedding layer)

In [0]:
def embedding_out(X):

  max_len = 10
  embedding_out = np.zeros((X.shape[0],max_len,emb_dim)) 
  
  for ix in range(X.shape[0]): 
     # to go to every sentence
     X[ix] = X[ix].split()

     for ij in range(len(X[ix])): 
       # to go every word in current (ix) sentence 

       try:
         embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]

       except:
         embedding_out[ix][ij] = np.zeros((50,))

  return embedding_out

embeddings_matrix_train = embedding_out(X_train)  
embeddings_matrix_test = embedding_out(X_test)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
embeddings_matrix_train.shape,embeddings_matrix_test.shape

((132, 10, 50), (56, 10, 50))

### Define the RNN/LSTM Model

In [0]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

from keras.datasets import mnist
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.compat.v1.keras.optimizers import Adam

import matplotlib.pyplot as plt
import math

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_6 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [0]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_accuracy',patience=5)

hist = model.fit(embeddings_matrix_train,y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 0s 5ms/sample - loss: 1.6089 - accuracy: 0.2190 - val_loss: 1.6096 - val_accuracy: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 231us/sample - loss: 1.5506 - accuracy: 0.3238 - val_loss: 1.6179 - val_accuracy: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 224us/sample - loss: 1.5411 - accuracy: 0.3333 - val_loss: 1.6312 - val_accuracy: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 255us/sample - loss: 1.4839 - accuracy: 0.3905 - val_loss: 1.6403 - val_accuracy: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 210us/sample - loss: 1.4855 - accuracy: 0.3810 - val_loss: 1.6494 - val_accuracy: 0.2222
Epoch 6/100
105/105 [==============================] - 0s 236us/sample - loss: 1.4563 - accuracy: 0.3619 - val_loss: 1.6556 - val_accuracy: 0.2222
Epoch 7/100
105/105 [==============================] - 0s 230us/sample - lo

In [0]:
pred = model.predict_classes(embeddings_matrix_test)

In [0]:
print(pred)

[4 3 2 2 2 2 3 2 2 2 1 2 0 3 1 3 2 2 3 2 0 0 4 2 3 3 2 0 1 2 0 1 0 2 0 1 2
 4 4 2 1 0 0 2 2 2 2 2 0 1 1 0 3 2 2 0]


In [0]:
model.evaluate(embeddings_matrix_test,y_test)

[1.4346551895141602, 0.71428573]

In [0]:
model.save("best_model.h5")

In [0]:
model.load_weights("best_model.h5")

In [0]:
for i in range(30):
  print(' '.join(X_test[i]))
  print(emoji.emojize(emoji_dict[str(np.argmax(y_test[i]))]))
  print(emoji.emojize(emoji_dict[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a very nice raise
😃
😃
she got me a nice present
😃
😃
ha ha ha it was so funny
😃
😃
he is a good friend
😃
😃
I am upset
😞
😞
We had such a lovely dinner tonight
😃
😃
where is the food
🍴
😃
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
💗
are you serious
😞
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop pissing me off
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
💗
💗
you brighten my day
😃
💗
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
give me the ball
⚾
😃
My grandmother is the love of my life
💗
💗
enjoy your game
⚾
⚾
valentine day is near
😃
😃
